In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# importing model and utils from xgbse
from xgbse import XGBSEKaplanNeighbors, XGBSEDebiasedBCE, XGBSEKaplanTree, XGBSEBootstrapEstimator
from xgbse.converters import convert_to_structured

# Building a survival model

Using Loft package for survival analyses on maintanance dataset

https://github.com/loft-br/xgboost-survival-embeddings


https://towardsdatascience.com/xgbse-improving-xgboost-for-survival-analysis-393d47f1384a

## Reading Data

In [2]:
#training data
col_names = ['asset_id', 'runtime', 'setting1', 'setting2', 'setting3', 'tag1', 'tag2', 'tag3', 'tag4', 'tag5', 'tag6', 
             'tag7', 'tag8', 'tag9', 'tag10', 'tag11', 'tag12', 'tag13', 'tag14', 'tag15','tga16', 'tag17',
            'tag18', 'tag19', 'tag20','tag21','tag22','tag23']
drop_columns = ['tag22','tag23']

df = pd.read_csv('./data/PM_train.txt', sep=' ', header=None)
df.columns = col_names

df.drop(columns=drop_columns, inplace=True)

#test data
df_test = pd.read_csv('./data/PM_test.txt', sep=' ', header=None)
df_test.columns = col_names
df_test.drop(columns=drop_columns,inplace=True)
df_test.head()

,asset_id,runtime,setting1,setting2,setting3,tag1,tag2,tag3,tag4,tag5,...,tag12,tag13,tag14,tag15,tga16,tag17,tag18,tag19,tag20,tag21
0,1,1,0.0023,0.0003,100.0,518.67,643.02,1585.29,1398.21,14.62,...,521.72,2388.03,8125.55,8.4052,0.03,392,2388,100.0,38.86,23.3735
1,1,2,-0.0027,-0.0003,100.0,518.67,641.71,1588.45,1395.42,14.62,...,522.16,2388.06,8139.62,8.3803,0.03,393,2388,100.0,39.02,23.3916
2,1,3,0.0003,0.0001,100.0,518.67,642.46,1586.94,1401.34,14.62,...,521.97,2388.03,8130.10,8.4441,0.03,393,2388,100.0,39.08,23.4166
3,1,4,0.0042,0.0000,100.0,518.67,642.44,1584.12,1406.42,14.62,...,521.38,2388.05,8132.90,8.3917,0.03,391,2388,100.0,39.00,23.3737
4,1,5,0.0014,0.0000,100.0,518.67,642.51,1587.19,1401.92,14.62,...,522.15,2388.03,8129.54,8.4031,0.03,390,2388,100.0,38.99,23.4130


## Creating RUL 

In [3]:
def format_data(df):
    df_asset_rul = df.groupby('asset_id').max()['runtime']
    df_asset_rul.name = 'max_life_span'

    df = pd.merge(df, df_asset_rul, on='asset_id')
    df['rul'] = df['max_life_span']-df['runtime']
    df['almost_failing'] = df['rul']<20
    return df

df = format_data(df)
df.head()

,asset_id,runtime,setting1,setting2,setting3,tag1,tag2,tag3,tag4,tag5,...,tag15,tga16,tag17,tag18,tag19,tag20,tag21,max_life_span,rul,almost_failing
0,1,1,-0.0007,-0.0004,100.0,518.67,641.82,1589.70,1400.60,14.62,...,8.4195,0.03,392,2388,100.0,39.06,23.4190,192,191,False
1,1,2,0.0019,-0.0003,100.0,518.67,642.15,1591.82,1403.14,14.62,...,8.4318,0.03,392,2388,100.0,39.00,23.4236,192,190,False
2,1,3,-0.0043,0.0003,100.0,518.67,642.35,1587.99,1404.20,14.62,...,8.4178,0.03,390,2388,100.0,38.95,23.3442,192,189,False
3,1,4,0.0007,0.0000,100.0,518.67,642.35,1582.79,1401.87,14.62,...,8.3682,0.03,392,2388,100.0,38.88,23.3739,192,188,False
4,1,5,-0.0019,-0.0002,100.0,518.67,642.37,1582.85,1406.22,14.62,...,8.4294,0.03,393,2388,100.0,38.90,23.4044,192,187,False


## Basic Cleaning

In [4]:
def basic_clean(df, remove_tags=None, drop_asset_aux=True):
    if remove_tags is None:
        df_filter = df.describe().loc['std']==0
        df_filter = df_filter.where(df_filter == True)
        remove_tags = df_filter.dropna().index.to_list()

    df.drop(columns = remove_tags, inplace=True)
    if drop_asset_aux:
        df.drop(columns = ['max_life_span','asset_id','rul'], inplace=True) # not usefull information or leakage remove
    
    return df, remove_tags

df, remove_tags = basic_clean(df)
remove_tags

['setting3', 'tag1', 'tag10', 'tag18', 'tag19']

In [5]:
df.head()

,runtime,setting1,setting2,tag2,tag3,tag4,tag5,tag6,tag7,tag8,...,tag11,tag12,tag13,tag14,tag15,tga16,tag17,tag20,tag21,almost_failing
0,1,-0.0007,-0.0004,641.82,1589.70,1400.60,14.62,21.61,554.36,2388.06,...,47.47,521.66,2388.02,8138.62,8.4195,0.03,392,39.06,23.4190,False
1,2,0.0019,-0.0003,642.15,1591.82,1403.14,14.62,21.61,553.75,2388.04,...,47.49,522.28,2388.07,8131.49,8.4318,0.03,392,39.00,23.4236,False
2,3,-0.0043,0.0003,642.35,1587.99,1404.20,14.62,21.61,554.26,2388.08,...,47.27,522.42,2388.03,8133.23,8.4178,0.03,390,38.95,23.3442,False
3,4,0.0007,0.0000,642.35,1582.79,1401.87,14.62,21.61,554.45,2388.11,...,47.13,522.86,2388.08,8133.83,8.3682,0.03,392,38.88,23.3739,False
4,5,-0.0019,-0.0002,642.37,1582.85,1406.22,14.62,21.61,554.00,2388.06,...,47.28,522.19,2388.04,8133.80,8.4294,0.03,393,38.90,23.4044,False


## Training

In [6]:
# importing model and utils from xgbse
from xgbse import XGBSEKaplanNeighbors
from xgbse.converters import convert_to_structured

# getting data

# splitting to X, y format
X = df#.head(2000)
y = convert_to_structured(X['runtime'], X['almost_failing'])
X = X.drop(['almost_failing'], axis=1)

# fitting xgbse model
xgbse_model = XGBSEKaplanNeighbors(n_neighbors=50)
#xgbse_model = XGBSEDebiasedBCE()
#xgbse_model = XGBSEKaplanTree()
xgbse_model.fit(X, y)

# predicting
event_probs = xgbse_model.predict(X)
event_probs.head(180)

,110,131,152,173,194,215,236,257,278,299,320,341
0,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
1,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
2,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
3,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
4,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
175,1.0,1.000000,0.976744,0.976744,0.850291,0.680233,0.680233,0.680233,0.680233,0.680233,0.680233,0.680233
176,1.0,1.000000,0.920000,0.571261,0.218739,0.124994,0.093745,0.093745,0.093745,0.000000,0.000000,0.000000
177,1.0,1.000000,0.918367,0.708817,0.393625,0.262291,0.196718,0.196718,0.196718,0.000000,0.000000,0.000000
178,1.0,1.000000,0.955511,0.923660,0.577294,0.505132,0.404106,0.404106,0.202053,0.202053,0.202053,0.202053


## Testing the model

In [7]:
df_test = format_data(df_test)
df_test, remove_tags = basic_clean(df_test, remove_tags, False)
df_test.head()

,asset_id,runtime,setting1,setting2,tag2,tag3,tag4,tag5,tag6,tag7,...,tag13,tag14,tag15,tga16,tag17,tag20,tag21,max_life_span,rul,almost_failing
0,1,1,0.0023,0.0003,643.02,1585.29,1398.21,14.62,21.61,553.90,...,2388.03,8125.55,8.4052,0.03,392,38.86,23.3735,31,30,False
1,1,2,-0.0027,-0.0003,641.71,1588.45,1395.42,14.62,21.61,554.85,...,2388.06,8139.62,8.3803,0.03,393,39.02,23.3916,31,29,False
2,1,3,0.0003,0.0001,642.46,1586.94,1401.34,14.62,21.61,554.11,...,2388.03,8130.10,8.4441,0.03,393,39.08,23.4166,31,28,False
3,1,4,0.0042,0.0000,642.44,1584.12,1406.42,14.62,21.61,554.07,...,2388.05,8132.90,8.3917,0.03,391,39.00,23.3737,31,27,False
4,1,5,0.0014,0.0000,642.51,1587.19,1401.92,14.62,21.61,554.16,...,2388.03,8129.54,8.4031,0.03,390,38.99,23.4130,31,26,False


In [8]:
df_last_frame = df_test.groupby('asset_id').last()
df_last_frame_copy = df_last_frame.reset_index().copy()
df_last_frame.drop(columns=['max_life_span','rul','almost_failing'], inplace=True)
df_last_frame.head(2)

,runtime,setting1,setting2,tag2,tag3,tag4,tag5,tag6,tag7,tag8,tag9,tag11,tag12,tag13,tag14,tag15,tga16,tag17,tag20,tag21
asset_id,,,,,,,,,,,,,,,,,,,,
1,31,-0.0006,0.0004,642.58,1581.22,1398.91,14.62,21.61,554.42,2388.08,9056.40,47.23,521.79,2388.06,8130.11,8.4024,0.03,393,38.81,23.3552
2,49,0.0018,-0.0001,642.55,1586.59,1410.83,14.62,21.61,553.52,2388.10,9044.77,47.67,521.74,2388.09,8126.90,8.4505,0.03,391,38.81,23.2618


In [9]:
event_probs = xgbse_model.predict(df_last_frame)

In [10]:
MAX_RUL = 1000
def get_class(row):
    ret = row.where(row<0.5).dropna()
    if len(ret)==0:
        return MAX_RUL
    else:
        failed_time =  row.where(row<0.5).dropna().index[0]
        return failed_time
        

test_result = event_probs
test_result['runtime'] = df_last_frame_copy['runtime']
test_result['max_rul'] = test_result.apply(get_class, axis=1)
test_result['class'] = test_result['max_rul'] - test_result['runtime'] - 20 
#if is bellow zero it will be a failure in up to 20 cycles

In [11]:
df_truth = pd.read_csv('.\data\gabarito_PM_truth.txt',header=None,names=['truth'])
test_result['truth'] = df_truth['truth']

test_result['truth_class'] = test_result['truth']<=20 #falha em rul menor que 20 ciclos
test_result['class_pred'] = test_result['class']<=0

In [12]:
from sklearn.metrics import classification_report
print(classification_report(test_result['truth_class'], test_result['class_pred']))

              precision    recall  f1-score   support

       False       0.92      0.99      0.95        84
        True       0.90      0.56      0.69        16

    accuracy                           0.92       100
   macro avg       0.91      0.78      0.82       100
weighted avg       0.92      0.92      0.91       100



## Baseline to compare the model

In [13]:
#using standard xgboost
from xgboost import XGBClassifier
X = df
y = X['almost_failing']
X = X.drop(['almost_failing'], axis=1)
baseline_model = XGBClassifier()
baseline_model.fit(X, y)

[20:46:48] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\RMANSUR\AppData\Roaming\Python\Python37\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=4, num_parallel_tree=1,
              objective='binary:logistic', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', use_label_encoder=True,
              validate_parameters=1, verbosity=None)

In [14]:
df_test['baseline_class'] = baseline_model.predict(df_test[X.columns])
df_resul_baseline = df_test.groupby('asset_id').last()

In [15]:
print(classification_report(test_result['truth_class'], df_resul_baseline['baseline_class']))

              precision    recall  f1-score   support

       False       0.98      0.98      0.98        84
        True       0.88      0.88      0.88        16

    accuracy                           0.96       100
   macro avg       0.93      0.93      0.93       100
weighted avg       0.96      0.96      0.96       100



In [ ]:
#Is not better than standard xgboost (straight forward approach without tunning) but the capacity to generate 
#a curve to be analysed can be usefull.